# runalyze2video
```markdown
Autor:          Maik 'Schrottie' Bischoff
Beschreibung:   Erzeugen von (vertikalen) Statistikvideos aus Daten von runalyze.com.
Version:        0.6
Stand:          28.03.2024
```

### Importe

In [ ]:
import requests
#from bs4 import BeautifulSoup
from datetime import datetime, timedelta, timezone
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from moviepy.editor import *
import numpy as np
import pandas as pd
from moviepy.config import change_settings
import cv2
import shutil
import os
from dotenv import load_dotenv
import re
import sqlite3
from PIL import Image, ImageDraw, ImageFont
import calendar
import warnings
import locale

locale.setlocale(locale.LC_ALL, 'de_DE') # Setzen des Gebietsschemas auf Deutsch
warnings.filterwarnings("ignore", category=DeprecationWarning) # Ignoriere DeprecationWarning
warnings.filterwarnings("ignore", category=FutureWarning) # Ignoriere FutureWarning
warnings.filterwarnings("ignore", category=UserWarning) # Ignoriere UserWarning

### Globale Variablen und andere Einstellungen

In [ ]:
sportid = '800522' # 800522 = Laufen
run_mode = 2 # 1 letzte Kalenderwoche, 2 letzter Kalendermonat, 3 Benutzerdefinierter Bereich, Start und Ende erforderlich!
start_date_string = '2024-03-25'
end_date_string = '2024-03-29'
end_date = None # '2024-03-21'
max_download_frequency = 23 # Maximal alle 23 Stunden neue Daten laden. Geringere Werte möglich, aber bitte mit Vorsicht 
                            # nutzen um zu hohe Last bei runalyze.com zu vermeiden!

# Falls ImageMagick nicht installiert werden kann, einfach eine portable Version
# herunterladen und hier den Pfad angeben
#change_settings({"IMAGEMAGICK_BINARY": r"D:\Imagemagick\convert.exe"})

# Diverse Variablen
chart_file_pic = 'tmp/chart.png' # Temporäres Datendiagramm
save_chart = True # Soll das Diagramm gespeichert werden?
csv_file = 'tmp/activities.csv' # Name der heruntergeladenen Daten
db_filename = 'runalyze_data.db' # Name der zu verwendenden SQLite-Datenbank
table_name = 'all_data' # Name der Datentabelle

# Video
duration_per_row = 1  # Dauer der Anzeige einzelner Aktivitäten im Video in sec
fade_duration = 1  # Dauer des Überganges in sec
final_duration = 5  # Dauer der Anzeige der Gesamtdaten in sec
video_intro_png = 'stuff/intro_photo.png' # Optional: PNG das für das Intro verwendet werden soll 
                                    # (1920x1080, min 250px breiter, freier Bereich mittig auf den oberen 1725px erfordrlich)
intro_clip_text = 'Schrotties Laufstatistik' # Text für das Intro
intro_clip_line3 = '– created with runalyze2video –' # Credits-Text für das Intro
intro_clip_duration = 4 # Anzeigedauer des Introbildes in sec
video_intro_file = 'tmp/intro.png' # Dateiname für das temporäre Introbild
video_outro_png = 'stuff/outro.png' # # Optional: PNG das für ein Outro verwendet werden soll 
final_video_clip = None # Nicht ändern, wird später automatisch angepasst!
final_chart_pic = None # Nicht ändern, wird später automatisch angepasst!
outro_clip_duration = 3 # Dauer eines (optionalen) Outro
fps = 30 # Videoframerate

# Text im Video formatieren
intro_clip_fontsize_normal = 85 # Schriftgröße für den Introtext
intro_clip_fontsize_bold = 130 # Schriftgröße für den Datumsbereich im Intro
x_image_position = 150 # Position des eingefügten Textes auf dem Introbild (Abstand von links)
y_image_position = 20 # Position des eingefügten Textes auf dem Introbild (Abstand von oben)
intro_clip_fontcolor = (0, 0, 0) # Textfarbe auf dem Intro (Achtung: RGB!)
intro_clip_fontcolor_credits = (71, 147, 64)
font = cv2.FONT_HERSHEY_TRIPLEX # Schriftart
font_scale = 3 # Textgröße
font_color = (107, 82, 68)  # Textfarbe für einzelne Aktivitäten (Achtung: BGR!)
sum_font_color = (79, 40, 163) # Textfarbe für die Zusammenfassung (Achtung: BGR!)
font_thickness = 7 # Textdicke
ttf_file_normal = 'stuff/DroidSans.ttf' # Schriftart normal (für Introtext)
ttf_file_bold = 'stuff/DroidSans-Bold.ttf' # Schriftart fett (für Introtext)

# r_type zu richtigen Aktivitäten mappen
activity_type_map = {
    994850.0: 'Easy Run',
    994851.0: 'Fahrtspiel',
    994852.0: 'Intervalltraining',
    994853.0: 'Tempodauerlauf',
    994854.0: 'Wettkampf',
    994855.0: 'Regenerationslauf',
    994856.0: 'Longrun'
}
catch_all_activity = 'Hallenhalma' # Wenn der Aktivitätstyp nicht im Mapping enthalten ist, kommt dieser hier rein

### Funktion: get_last_fetch()
```markdown
Funktion zur Überprüfung des Zeitpunktes des letzten Datendownload - Liegt
dieser weniger als Stunden zurück, als in 'max_download_frequency' festgelegt,
erfolgt kein Download neuer Daten vom JSON-Endpunkt.

In [ ]:
def get_last_fetch():
    # SQLite-Verbindung herstellen
    conn = sqlite3.connect(db_filename)
    cursor = conn.cursor()

    # Prüfen, ob die Tabelle bereits existiert
    cursor.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='last_fetch'")
    table_exists = cursor.fetchone()

    if not table_exists:
        # Wird die Tabelle 'last_fetch' nicht gefunden, dann erstellen und Zeitstempel einwerfen
        cursor.execute("CREATE TABLE IF NOT EXISTS last_fetch (date TEXT)")
        current_time = datetime.now().isoformat()
        cursor.execute("INSERT INTO last_fetch (date) VALUES (?)", (current_time,))
        conn.commit()
        return 1000  # Rückgabewert 1000, damit der Download initiiert wird
    else:
        # Gibt es die Tabelle, dann jüngsten Wert nehmen und ermitteln, wieviele Stunden seither vergangen sind
        cursor.execute("SELECT date FROM last_fetch ORDER BY date DESC LIMIT 1")
        last_fetch = cursor.fetchone()
        last_fetch_date = datetime.fromisoformat(last_fetch[0])
        current_time = datetime.now()
        age = current_time - last_fetch_date
        age_hours = int(age.total_seconds() / 3600)
        return age_hours

    # Verbindung schließen
    cursor.close()
    conn.close()

### Funktion: login_with_username_password()
```markdown
Initiiert den Login bei Runalyze, damit der Datendownload am JSON-Endpunkt auch
Daten bringt. Diese werden nur an eingeloggte Benutzer herausgegeben.
```

In [ ]:
def login_with_username_password():
    
    load_dotenv()
    username = os.getenv('RUNALYZE_USERNAME')
    password = os.getenv('RUNALYZE_PASSWORD')
    
    login_url = 'https://runalyze.com/login'
    session = requests.Session()
    
    # HTML-Inhalt der Login-Seite holen
    response = session.get(login_url)
    html_content = response.text
    
    # CSRF-Token extrahieren
    csrf_token_match = re.search(r'<input[^>]*name=["\']?_csrf_token["\']?[^>]*value=["\']?([^"\'>\s]+)', html_content)

    if csrf_token_match:
        csrf_token = csrf_token_match.group(1)
    else:
        print("CSRF-Token konnte nicht gefunden werden.")
        return None
    
    # Anmelden
    payload = {
        '_username': username,
        '_password': password,
        '_remember_me' : 'off',
        '_csrf_token': csrf_token
    }
    response = session.post(login_url, data=payload)

    # Überprüfen, ob die Anmeldung erfolgreich war
    if response.status_code == 200:
        print("Anmeldung erfolgreich!")
        return session
    else:
        print(f"Anmeldung fehlgeschlagen. Statuscode: {response.status_code}")
        return None

### Funktion: fetch_activity_data_csv(arg)
```markdown
Holt die CSV-Datei mit den Aktivitätsdaten vom JSON-Endpunkt ab. Die Datei
enthält alle bei Runalyze für den eingeloggten Benutzer hinterlegten Daten.
Somit wird stets ein vollständiges Paket geholt.
```

In [ ]:
def fetch_activity_data_csv(session):
    csv_url = 'https://runalyze.com/_internal/data/activities/all'
    
    # Die CSV-Datei mit allen Daten abholen
    response = session.get(csv_url)
    
    # Überprüfen, ob der Abruf erfolgreich war
    if response.status_code == 200:
        with open(csv_file, 'wb') as f:
            f.write(response.content)
        print("CSV-Datei erfolgreich heruntergeladen.")
    else:
        print("Fehler beim Abrufen der CSV-Datei.")

### Funktion: import_csv_to_sqlite()
```markdown
Schreibt die Daten aus dem CSV in eine SQLite-Datenbank. Wenn darin noch keine 
entsprechende Tabelle existiert, wird diese angelegt und befüllt.
Existiert sie schon, wird ein Backup daon gefertigt und anschließend alle Daten
neu eingekippt. Sollte schon ein älteres Backup bestehen, wird es zuvor gelöscht
und dann durch das Neue ersetzt.
```

In [ ]:
def import_csv_to_sqlite():
    # Daten aus CSV-Datei lesen
    df = pd.read_csv(csv_file)

    # SQLite-Datenbankverbindung herstellen
    conn = sqlite3.connect(db_filename)
    cursor = conn.cursor()

    # Prüfen, ob die Tabelle bereits existiert
    cursor.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table_name}'")
    table_exists = cursor.fetchone()

    # Wenn die Tabelle existiert, sichere sie als 'bckp_table_name' und lösche sie
    if table_exists:
        backup_table_name = f'bckp_{table_name}'
        # Prüfen, ob die Backup-Tabelle existiert und ggf. löschen
        cursor.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{backup_table_name}'")
        backup_table_exists = cursor.fetchone()
        if backup_table_exists:
            cursor.execute(f"DROP TABLE IF EXISTS {backup_table_name}")

        # Tabelle umbenennen und löschen
        cursor.execute(f"ALTER TABLE {table_name} RENAME TO {backup_table_name}")  # Sicherung der Tabelle
        cursor.execute(f"DROP TABLE IF EXISTS {table_name}")  # Tabelle löschen

    # Tabelle aus dem DataFrame erstellen
    df.to_sql(table_name, conn, index=False)

    # Zeitstempel setzen, um zu häufiges Herunterladen der CSV zu vermeiden
    cursor.execute("CREATE TABLE IF NOT EXISTS last_fetch (date TEXT)")
    current_time = datetime.now().isoformat()
    cursor.execute("INSERT INTO last_fetch (date) VALUES (?)", (current_time,))

    # Transaktion bestätigen und Verbindung schließen
    conn.commit()
    conn.close()

### Funktion: query_data_from_db()
```markdown
Sammelt die relevanten Daten aus der SQLite-Datenbank und schreibt sie in ein
Pandas-Dataframe. Gleichzeitig werden noch ein paar Variablen gestrickt, die
im weiteren Verlauf erforderlich sein werden.
```

In [ ]:
def query_data_from_db():
    # Mapping für die Spalten zwischen Datenbank und DataFrame
    column_mapping = {
        'time': 'date',
        'sportid': 'a_type',
        'typeid': 'r_type',
        'distance': 'distance',
        's': 'duration',
        'title': 'title'
    }
    
    global start_date
    global end_date
    # SQLite-Datenbankverbindung herstellen
    conn = sqlite3.connect(db_filename)
    
    # Spalten für die SQL-Abfrage aus dem Mapping extrahieren
    selected_columns = ', '.join(column_mapping.keys())
    # Zeitzone UTC
    utc_timezone = timezone.utc

    # Parameter für die SQL-Abfrage bauen
    if run_mode == 1:
        today_utc = datetime.now(utc_timezone)
        start_date = int((today_utc - timedelta(days=today_utc.weekday() + 7)).replace(hour=0, minute=0, second=0, microsecond=0).timestamp())
        end_date = int((today_utc - timedelta(days=today_utc.weekday())).replace(hour=0, minute=0, second=0, microsecond=0).timestamp())
    elif run_mode == 2:
        today_utc = datetime.now(utc_timezone)
        first_day_of_last_month = today_utc.replace(day=1) - timedelta(days=2)
        start_date = int((first_day_of_last_month.replace(day=1)).replace(hour=0, minute=0, second=0, microsecond=0).timestamp())
        end_date = int(today_utc.replace(day=1, hour=0, minute=0, second=0, microsecond=0).timestamp())
    elif run_mode == 3:
        start_date = int(datetime.strptime(start_date_string, '%Y-%m-%d').timestamp())
        end_date = int(datetime.strptime(end_date_string, '%Y-%m-%d').timestamp())
    else:
        print('Query failed!')
        return None  # Rückgabe None bei ungültigem run_mode
        
    # SQL-Abfrage für den Datenabruf
    query = f"""
        SELECT {selected_columns}
        FROM all_data
        WHERE sportid = {sportid} AND time >= '{start_date}' AND time <= '{end_date}'
    """

    # Daten aus der Datenbank abrufen
    df = pd.read_sql_query(query, conn)
    
    # Felder entsprechend dem Mapping umbenennen
    df.rename(columns=column_mapping, inplace=True)

    # Jetzt die Pace berechnen.
    df["pace"] = df["duration"] / df["distance"]
    df["pace"] = df["pace"] / 60
    df["pace"] = df["pace"].apply(lambda x: f"{int(x // 1):02d}:{int(x % 1 * 60):02d}")

    # Umwandlung der Spalte "duration" von Sekunden in Zeitformat (hh:mm:ss)
    df['duration'] = df['duration'].apply(lambda x: datetime.utcfromtimestamp(x).strftime('%H:%M:%S'))
    df['date_timestamp'] = pd.to_datetime(df['date'], unit='s')
    # Neuen Namen für finalen Videoclip und finales Diagramm erzeugen
    start_date = datetime.utcfromtimestamp(start_date)
    end_date = datetime.utcfromtimestamp(end_date)
    print(start_date)
    print(end_date)
    min_date_str = start_date.strftime('%Y%m%d')
    # Einen Tag vom end_date abziehen, damit die Dateinamen passen
    end_date -= timedelta(days=1)
    # end_date in das gewünschte Format konvertieren
    max_date_str = end_date.strftime('%Y%m%d')
    global final_video_clip
    final_video_clip = f"movies/activity_movie_{min_date_str}_{max_date_str}.mp4"
    global final_chart_pic
    final_chart_pic = f"movies/chart_{min_date_str}_{max_date_str}.png"
    # Transaktion bestätigen und Verbindung schließen
    conn.commit()
    conn.close()
    # print(df.dtypes) # DEBUG!
    # print(df) # DEBUG!
    return df

### Funktion: edit_intro_image()
```markdown
Funktion zum Bearbeiten des Intro-Bildes. Hier wird das (optionale) Introbild
mit einem Text versehen.
```

In [ ]:
def edit_intro_image():
    # Laden des PNG-Bildes
    intro_image = Image.open(video_intro_png)
    intro_breite, intro_höhe = intro_image.size

    # Erstellen der ersten Textgrafik
    text_image1 = create_text_image(intro_clip_text, intro_clip_fontsize_bold, intro_clip_fontcolor, 1700, 125)

    # Erstellen der zweiten Textgrafik
    date_text = f"{start_date.strftime('%d.%m.%Y')} bis {end_date.strftime('%d.%m.%Y')}"
    text_image2 = create_text_image(date_text, intro_clip_fontsize_bold, intro_clip_fontcolor, 1700, 125)

    # Erstellen der dritten Textgrafik
    if run_mode == 1:
        calendar_week = end_date.isocalendar()[1]
        intro_clip_line3 = f"Statistik für die {calendar_week}. Kalenderwoche"
    elif run_mode == 2:
        intro_clip_line3 = f"Statistik für den Monat {end_date.strftime('%B')}"

    text_image3 = create_text_image(intro_clip_line3, intro_clip_fontsize_normal, intro_clip_fontcolor_credits, 1700, 125)
    
    # Textbilder um 90 Grad drehen
    text_image1 = text_image1.rotate(90, expand=True)
    text_image2 = text_image2.rotate(90, expand=True)
    text_image3 = text_image3.rotate(90, expand=True)

    # Zusammenfügen der Textbilder zu einer kombinierten Grafik
    combined_text_image = Image.new('RGBA', (375, 1700), (255, 255, 255, 0))
    combined_text_image.paste(text_image1, (0, 0))
    combined_text_image.paste(text_image2, (125, 0))
    combined_text_image.paste(text_image3, (250, 0))

    # Grafik um 15 Grad nach rechts drehen
    combined_text_image = combined_text_image.rotate(-15, expand=True)

    # Grafik in das Ursprungsbild einfügen
    intro_image.paste(combined_text_image, (x_image_position, y_image_position), combined_text_image)

    # Bearbeitetes Bild speichern
    intro_image.save(video_intro_file)

### Funktion: create_text_image(args)
```markdown
Erzeugt die Textgrafiken, die für das Intro-Image benötigt werden. (Kann auch
mit mehreren Zeilen umgehen, das wird aber aktuell "aus Gründen" nicht benötigt.)
```

In [ ]:

def create_text_image(text, font_size, font_color, width, height):
    # Erstellen eines Zeichenobjekts für den Text
    text_image = Image.new('RGBA', (width, height), (255, 255, 255, 0))
    draw = ImageDraw.Draw(text_image)

    # Definieren der Schriftart und -größe
    font = ImageFont.truetype(ttf_file_normal, font_size)

    # Text auf das Textbild zeichnen
    text_width = draw.textlength(text, font=font)
    lines = len(text.split('\n'))  # Anzahl der Zeilen zählen (einschließlich potenzieller Zeilenumbrüche)
    text_height = font.size * lines  # Schriftgröße mit der Anzahl der Zeilen multiplizieren

    draw.text(((width - text_width) / 2, (height - text_height) / 2), text, fill=font_color, font=font)

    return text_image

### Funktion: create_chart(arg)
```markdown
Erzeugt ein Balkendiagramm mit den Daten der Aktivitäten des zu nutzenden Zeitraumes
(je nach run_mode). Dabei stellt die Distanz die Balkenlänge dar und auf dem Balken
wird die DAuer und die Pace angezeigt.
```

In [ ]:
def create_chart(activities):
    # Anzahl der Zeilen im DataFrame
    n_rows = activities.shape[0]

    # Erstellen eines Figurenobjekts mit quadratischen Dimensionen
    fig, ax = plt.subplots(figsize=(10, 10))

    # Beschriftung der Y-Achse auf "Von - Bis" setzen
    min_date = datetime.utcfromtimestamp(activities['date'].min())
    max_date = datetime.utcfromtimestamp(activities['date'].max())
    min_date_str = min_date.strftime('%d.%m.%Y')
    max_date_str = max_date.strftime('%d.%m.%Y')
    ax.set_ylabel(f"{min_date_str} - {max_date_str}")

    # Erstellen eines horizontalen Balkendiagramms für die Distanz
    for i, (dist, dur, pace) in enumerate(zip(activities["distance"], activities["duration"], activities["pace"])):
        
        # Farbe für die Balken setzen
        color = mcolors.to_rgba_array(plt.cm.bone(0.3 + i * 0.3 / n_rows))

        # Die Balken zeichnen
        ax.barh(i, dist, color=color, edgecolor='black')
        
        # Formatierung der Dauer, damit kein "0 days" vorangestellt ist
        dur_formatted = str(dur).split()[-1]

        # Die Balken mit den Distanzwerten am rechten Rand des Balkens beschriften
        ax.text(dist, i, f"{dur_formatted} / {pace}  ", ha='right', va='center', rotation=0, color='white')

    # Beschriftung der X-Achse für die Distanz setzen
    ax.set_xlabel("Laufdistanz (km)")
    
    # Die Beschriftungen der Y-Achse entfernen
    ax.set_yticks([])
    
    # Gitterlinien im Hintergrund hinzufügen
    ax.grid(True, linestyle=':', linewidth=0.5)

    # Hintergrundfarbe setzen
    ax.set_facecolor('#fffff0')

    # Diagramm als Bild speichern
    fig.savefig(chart_file_pic, bbox_inches='tight', dpi=350)  
    
    # Das Diagramm anzeigen
    #plt.show()


### Funktion: create_video(arg)
```markdown
Erzeugt das finale Video. Ist eine Startgrafik angegeben, wird zunächst diese
angezeigt, dann folgen alle einzelnen Aktivitäten sowie am Ende eine Zusammenfassung
aller Daten.
```

In [ ]:
def create_video(df):

    # Höhe des Textbereichs und Bildbereichs berechnen
    text_area_height = 840
    image_area_height = 1080

    # Video-Writer-Objekt erstellen
    out = cv2.VideoWriter(final_video_clip, cv2.VideoWriter_fourcc(*'mp4v'), fps, (1080, 1920))

    # Überprüfen, ob ein Intro-Clip bereitgestellt und vorhanden ist
    if video_intro_png and os.path.exists(video_intro_file):
        # Intro-Clip laden und skalieren
        intro_clip = cv2.imread(video_intro_file)
        intro_clip_resized = cv2.resize(intro_clip, (1080, 1920))

        # Intro-Clip in Video schreiben
        for _ in range(int(fps * intro_clip_duration)):
            out.write(intro_clip_resized)

        # Leeres Frame für Übergang erstellen
        transition_frame = np.zeros((1920, 1080, 3), dtype=np.uint8)
        cv2.rectangle(transition_frame, (0, 0), (1080, 1920), (255, 255, 255), -1)  # Rahmen mit Weiß füllen

        # Alpha-Maske für Überblendung erstellen
        alpha_mask = np.linspace(0, 255, int(fps * fade_duration)).astype(np.uint8)

        # Alpha-Übergang auf Übergangsframe anwenden
        for alpha in alpha_mask:
            blended_frame = cv2.addWeighted(intro_clip_resized, 1 - alpha / 255, transition_frame, alpha / 255, 0)
            out.write(blended_frame)

    # Durch jeden Eintrag im DataFrame iterieren
    for index, row in df.iterrows():

        duration_str = str(row['duration'])

        date_str = row['date_timestamp'].strftime('%d.%m.%Y')

        # Laufart aus der Zuordnung erhalten oder den originalen Wert verwenden, falls nicht in der Zuordnung gefunden
        activity_type = activity_type_map.get(row['r_type'], catch_all_activity)

        # Text für die Zeile erstellen
        text_lines = [
            f"{date_str}",
            f"{activity_type}",
            f"{row['distance']} km",
            f"{duration_str}",
            f"{row['pace']} min/km"
        ]
        text = '\n'.join(text_lines)

        # Leeres Frame erstellen
        frame = np.zeros((1920, 1080, 3), dtype=np.uint8)
        cv2.rectangle(frame, (0, 0), (1080, 1920), (255, 255, 255), -1)  # Rahmen mit Weiß füllen

        # Text zum Frame hinzufügen
        text_size = cv2.getTextSize(text, font, font_scale, font_thickness)[0]
        text_x = (1080 - text_size[0]) // 2
        line_height = text_size[1] * 1.5  # Zeilenhöhe um das 1,5-fache erhöhen
        text_y = (text_area_height - line_height * len(text_lines)) // 2 + text_size[1]  # Text vertikal zentrieren
        for i, line in enumerate(text_lines):
            line_size = cv2.getTextSize(line, font, font_scale, font_thickness)[0]
            line_x = (1080 - line_size[0]) // 2
            line_y = int(text_y + i * line_height)  # Angepasste Zeilenhöhe verwenden
            cv2.putText(frame, line, (line_x, line_y), font, font_scale, font_color, font_thickness, cv2.LINE_AA)

        # Bild skalieren und zum Frame hinzufügen
        image_clip = cv2.imread(chart_file_pic)
        image_clip_resized = cv2.resize(image_clip, (1080, image_area_height))
        frame[text_area_height:, :] = image_clip_resized

        for _ in range(int(fps * duration_per_row)):
            out.write(frame)

        # Übergangsframe erstellen
        transition_frame = np.zeros((1920, 1080, 3), dtype=np.uint8)

        # Text zum Übergangsframe hinzufügen
        for i, line in enumerate(text_lines):
            line_size = cv2.getTextSize(line, font, font_scale, font_thickness)[0]
            line_x = (1080 - line_size[0]) // 2
            line_y = int(text_y + i * line_height)  # Angepasste Zeilenhöhe verwenden
            cv2.putText(transition_frame, line, (line_x, line_y), font, font_scale, font_color, font_thickness, cv2.LINE_AA)

        # Bild skalieren und zum Übergangsframe hinzufügen
        transition_frame[text_area_height:, :] = image_clip_resized
        cv2.rectangle(transition_frame, (0, 0), (1080, 840), (255, 255, 255), -1)  # Rahmen mit Weiß füllen

        # Alpha-Maske für Überblendung erstellen
        alpha_mask = np.linspace(0, 255, int(fps * fade_duration)).astype(np.uint8)

        # Alpha-Übergang für den Übergang anwenden
        for alpha in alpha_mask:
            blended_frame = cv2.addWeighted(frame, 1 - alpha / 255, transition_frame, alpha / 255, 0)
            out.write(blended_frame)

    # Gesamtstatistik erstellen
    # Distanz
    distance_sum = round(df['distance'].sum(), 2)
    # Dauer
    df['duration'] = pd.to_timedelta(df['duration'])
    df['duration_seconds'] = df['duration'].dt.total_seconds()
    duration_sum_seconds = df['duration_seconds'].sum()
    total_duration = timedelta(seconds=duration_sum_seconds)
    duration_sum = str(total_duration)
    # Pace (hier Durchschnitt anstelle von Summe)
    pace_mean = df['pace'].apply(lambda x: datetime.strptime(x, '%M:%S')).mean().strftime('%M:%S')

    summary_text_lines = [
        "Gesamt:",
        f"{distance_sum} km",
        f"{duration_sum}",
        f"{pace_mean} min/km" 
    ]
    summary_text = '\n'.join(summary_text_lines)
    summary_frame = np.zeros((1920, 1080, 3), dtype=np.uint8)
    cv2.rectangle(summary_frame, (0, 0), (1080, 1920), (255, 255, 255), -1)  # Rahmen mit Weiß füllen

    for i, line in enumerate(summary_text_lines):
        line_size = cv2.getTextSize(line, font, font_scale, font_thickness)[0]
        line_x = (1080 - line_size[0]) // 2
        line_y = int(text_y + i * line_height)  # Zeilenabstand anpassen
        cv2.putText(summary_frame, line, (line_x, line_y), font, font_scale, sum_font_color, font_thickness, cv2.LINE_AA)
        
    summary_frame[text_area_height:, :] = image_clip_resized

    for _ in range(int(fps * final_duration)):
        out.write(summary_frame)

    # Überprüfen, ob ein Outro-Clip bereitgestellt und vorhanden ist
    if video_outro_png and os.path.exists(video_outro_png):
        # Outro-Clip laden und skalieren
        outro_clip = cv2.imread(video_outro_png)
        outro_clip_resized = cv2.resize(outro_clip, (1080, 1920))

        # Leeres Frame für Übergang erstellen
        transition_frame = np.zeros((1920, 1080, 3), dtype=np.uint8)
        cv2.rectangle(transition_frame, (0, 0), (1080, 1920), (255, 255, 255), -1)  # Rahmen mit Weiß füllen

        # Alpha-Maske für Überblendung erstellen
        alpha_mask = np.linspace(0, 255, int(fps * fade_duration)).astype(np.uint8)

        # Alpha-Übergang auf Übergangsframe anwenden
        for alpha in alpha_mask:
            blended_frame = cv2.addWeighted(outro_clip_resized, alpha / 255, transition_frame, 1 - alpha / 255, 0)
            out.write(blended_frame)

        # Outro-Clip in Video schreiben
        for _ in range(int(fps * outro_clip_duration)):
            out.write(outro_clip_resized)

        # Alpha-Übergang auf Übergangsframe anwenden
        for alpha in alpha_mask[::-1]:
            blended_frame = cv2.addWeighted(outro_clip_resized, alpha / 255, transition_frame, 1 - alpha / 255, 0)
            out.write(blended_frame)

    # Video-Writer-Objekt freigeben
    out.release()

### Funktion: clean_up_tmp()
```markdown
Räumt im tmp-Verzeichnis auf, löscht also einfach alles was darin herumliegt. Vorab wird
geprüft, ob das Chartbild (Diagramm) gesichert werden soll und tut dies dann.
```

In [ ]:
def clean_up_tmp():
    # Soll das Chart gesichert werden?
    if save_chart:
        shutil.move(chart_file_pic, final_chart_pic)

    # Jetzt ab ins tmp-Verzeichnis und einmal kräftig durchfeudeln!
    os.chdir('tmp')
    for filename in os.listdir():
        os.remove(filename)
    # Und sicherheitshalber wieder zurück
    os.chdir('..')

### Hier fängt der ganze Spaß dann an …

In [ ]:
# Hauptfunktion
def main():
    age_in_hours = get_last_fetch()
    if age_in_hours is not None and age_in_hours > max_download_frequency:
        session = login_with_username_password()
        if session:
            # Alle Aktivitäten im CSV-Format abrufen
            fetch_activity_data_csv(session)
            # Alle neuen Aktivitäten in die Datenbank schreiben
            import_csv_to_sqlite()
    else:
        print(f"Die letzte Abfrage war vor weniger als {max_download_frequency} Stunden oder konnte nicht ermittelt werden.")

    # Dataframe vorbereiten
    activities = query_data_from_db()
    # Introbild vorbereiten/erzeugen
    edit_intro_image()
    # Diagramm zeichnen
    create_chart(activities)
    # Video erzeugen
    create_video(activities)
    # Aufräumen
    clean_up_tmp()
    
if __name__ == "__main__":
    main()